In [ ]:
import time
import pandas as pd
import pyarrow as pa

from shared_code.utility.spark.set_environ import set_azure_env
from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
set_azure_env()

In [ ]:
fs_adapter = AzureFileStorageAdapter('data')
file_system = fs_adapter.get_file_storage()

In [ ]:
%%time
extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(extant_data)

In [ ]:
%%time

columns = ['id', 'subreddit', 'title', 'caption', 'image_name', 'exists', 'curated']

slim_df = extant_data[columns]

exists = slim_df\
    .where(slim_df['exists'] == True)\
    .dropna(axis=1)\
    .dropna(how='all')\
    .reset_index(drop=True)

display(exists)

In [ ]:
%%time

foo = exists.rename(columns={'id':'Id', 'subreddit': 'SubReddit', 'title':"Title", 'caption': 'Caption', 'image_name': 'ImageName', 'exists': 'Exists', 'curated': 'Curated'})

display(foo)

In [ ]:
%%time

from shared_code.utility.storage.table import TableAdapter

tableAdapter: TableAdapter = TableAdapter()

curation_table = tableAdapter.get_table_client('curation')

records = foo.to_dict(orient='records')

for record in records:
    record['PartitionKey'] = record['SubReddit']
    record['RowKey'] = record['Id']
    curation_table.create_entity(entity=record)